# Lorenzo helper functions for data analysis

In [1]:
# Filter bad pixels
# def filter_bad_pixels(data, ratio):
    
#     """
#     Filters bad pixels in the input data based on noise level.

#     Parameters:
#         data (numpy.ndarray): Input data array of shape (rows, columns, frames).
#         ratio (float): Threshold ratio to identify bad pixels.

#     Returns:
#         tuple: A tuple containing the filtered data array and the number of identified bad pixels.
#     """
    
#     numBadPix = 0

#     noise = np.empty([np.shape(data)[0],np.shape(data)[1]])
#     for row, col in np.ndindex(np.shape(data)[0],np.shape(data)[1]):
#         noise[row,col] = np.std(data[row,col,:])

#     noise_median = np.median(noise)

#     for row, col in np.ndindex(np.shape(data)[0],np.shape(data)[1]):
#         if noise[row,col] > ratio*noise_median:
#             print('Found bad pixel {}:{}'.format(row,col))
#             data[row,col,:] = np.median(data)
#             numBadPix += 1

#     return data, numBadPix

# Filter bad pixels
def filter_bad_pixels(data, ratio):
    """
    Filters bad pixels in the input data based on noise level.

    Parameters:
        data (numpy.ndarray): Input data array of shape (rows, columns, frames).
        ratio (float): Threshold ratio to identify bad pixels.

    Returns:
        tuple: A tuple containing the filtered data array and the number of identified bad pixels.
    """
    # Create a copy of the input data array
    data_copy = data.copy()

    # Calculate noise for each pixel
    noise = np.std(data_copy, axis=2)
    noise_median = np.median(noise)

    # Identify and replace bad pixels
    mask = noise > ratio * noise_median
    data_copy[mask] = np.median(data_copy)

    num_bad_pixels = np.sum(mask)

    return data_copy, num_bad_pixels


# Reject outliers
def rejOutliers(data, m=2):
    return data[abs(data - np.mean(data)) < m * np.std(data)]

# Correlation matrix
def calculate_correlation(data):

    corr_mat   = np.empty([np.shape(data)[0],np.shape(data)[1]])
    
    for row, col in np.ndindex(np.shape(data)[0],np.shape(data)[1]):
        # Correlation coefficients
        corr_mat[row,col] = np.corrcoef(data[row,col,:], data[ref_row, ref_col,:])[0,1]

    corr_mat[ref_row,ref_col] = np.average(corr_mat)
    return corr_mat


# Configure charge injection
def chargeInjection(firstCol=120,LastCol=180,PulserValue=100,AsicNum=3):
    APP.prepareChargeInjection(AsicNum, firstCol, LastCol, PulserValue)

   
# Get Average Dark value over rows/cols
def getMedianDark(data):
    return np.median(np.median(data,axis=(0,1)))

# Charge injection ramp
def ChInjRamp(dataDebug, numberOfTriggers=3, Min=100, Max=150, numSteps=3):

    ChInjValues = np.linspace(start=Min, stop=Max, num=numSteps)
    PixelOutValues = np.empty(np.shape(ChInjValues))
      
    for i, PulserValue in enumerate(ChInjValues):
        # Get frames but look only at ref pixel
        data_temp = readoutFrames(dataDebug, numberOfTriggers,chargeInj=PulserValue)[ref_row,ref_col,:]
        # Get median across frames and add to results array
        data_temp = np.median(data_temp)
        PixelOutValues[i] = data_temp
    
    return ChInjValues, PixelOutValues
  
# Mask gain bit    
def clearB16(value):
    return value & ~(1 << 15)    

# Better functions !

In [2]:
# Charge injection ramp
def chargeInjRamp(dataDebug, AsicNum=1, Min=100, Max=150, numSteps=3):

    ChInjValues = np.linspace(start=Min, stop=Max, num=numSteps)
    PixelOutValues = np.empty(np.shape(ChInjValues))
      
    for i, PulserValue in enumerate(ChInjValues):
        # Get frames but look only at ref pixel
        chargeInjection(PulserValue=PulserValue, AsicNum=AsicNum)
        data_temp = readoutFrames(dataDebug, 1, chargeInj=PulserValue)[ref_row,ref_col,:]
        # Get median across frames and add to results array
        data_temp = np.median(data_temp)
        PixelOutValues[i] = data_temp
        #print(data_temp)
    
    return ChInjValues, PixelOutValues

In [1]:
# Readout frame(s)
def readoutFrames(dataDebug, numberOfTriggers=1, chargeInj=False):
    dataDebug.cleanData()

    for TrigNum in range(numberOfTriggers):
        if (chargeInj != False):
            chargeInjection(PulserValue=chargeInj)
        root.Trigger()
        print("{}".format(TrigNum+1), end='\r')
        time.sleep(0.01)

    # Although the triggers above may have finished, descrambling takes time
    while ( numberOfTriggers != dataDebug.getData().shape[2]) :
        time.sleep(0.1)
        print("Descrambled {}".format(dataDebug.getData().shape[2]), end='\r')
    print("Data Descrambled")
    print(dataDebug.getData().shape)

    # Data format = [row,col,frame_number]
    data = dataDebug.getData()
    dataDebug.cleanData()
    return data


# Timing functions

In [ ]:
# helper functions
# clock speed is 168MHz. 1 tick is 0.006 us
def setSR0(width, delay) :
    REGCTRL.SR0Delay1.set(delay)
    REGCTRL.SR0Width1.set(width)
    print(f'SR0 Width set to {REGCTRL.SR0Width1_us.get():.3f}uS')
    print(f'SR0 Delay set to {REGCTRL.SR0Delay_us.get():.3f}uS')
    
def setAcq1(width, delay) :
    REGCTRL.AcqDelay1.set(delay)
    REGCTRL.AcqWidth1.set(width)
    print(f'Acq1 Width set to {REGCTRL.AcqWidth1_us.get():.3f}uS')
    print(f'Acq1 Delay set to {REGCTRL.AcqDelay1_us.get():.3f}uS')
    
def setAcq2(width, delay) :
    REGCTRL.AcqDelay2.set(delay)
    REGCTRL.AcqWidth2.set(width)
    print(f'Acq2 Width set to {REGCTRL.AcqWidth2_us.get():.3f}uS')
    print(f'Acq2 Delay set to {REGCTRL.AcqDelay2_us.get():.3f}uS')    

def setR0(width, delay) :
    REGCTRL.R0Delay.set(delay)
    REGCTRL.R0Width.set(width)
    print(f'R0 Width set to {REGCTRL.R0Width_us.get():.3f}uS')
    print(f'R0 Delay set to {REGCTRL.R0Delay_us.get():.3f}uS') 

    
def setSync(width, delay) :
    REGCTRL.SyncDelay.set(delay)
    REGCTRL.SyncWidth.set(width)
    print(f'Sync Delay set to {REGCTRL.SyncDelay_us.get():.3f}uS')     

# Best settings
These are still w.i.p.

In [4]:
def setBestSettings(REGCTRL, ASIC0, ASIC1, ASIC2, ASIC3):

    intTime = 560 # 560 

    setR0(100+intTime*2, 200)
    setAcq1(intTime, intTime+200)

    # Print human-readable values
    print(f'-------------------------')
    baseline_int_time = REGCTRL.AcqDelay1_us.get() - REGCTRL.R0Delay_us.get()
    integration_time = REGCTRL.AcqWidth1_us.get()
    print(f'Baseline time:    {baseline_int_time} uS')
    print(f'Integration time: {integration_time} uS')
    print(f'Sampling done at: {REGCTRL.AcqWidth1_us.get()+REGCTRL.AcqDelay1_us.get()} uS')
    print(f'-------------------------')
    
    # Set ASIC registers
    ASIC1.mTest.set(False)
    ASIC2.mTest.set(False)
    ASIC1.CompTH_ePixM.set(0)
    ASIC2.CompTH_ePixM.set(0)
    ASIC1.Precharge_DAC_ePixM.set(45)
    ASIC2.Precharge_DAC_ePixM.set(45)
    ASIC1.DHg.set(False)
    ASIC2.DHg.set(False)

    # Optimal bias settings
    ASIC1.RefinN.set(3)
    ASIC2.RefinN.set(3)
    ASIC1.RefinP.set(0)
    ASIC2.RefinP.set(0)
    
    ASIC1.pipoclk_delay_row0.set(15)
    ASIC1.pipoclk_delay_row1.set(8)
    ASIC1.pipoclk_delay_row2.set(8)
    ASIC1.pipoclk_delay_row3.set(8)

    ASIC2.pipoclk_delay_row0.set(5)
    ASIC2.pipoclk_delay_row1.set(5)
    ASIC2.pipoclk_delay_row2.set(5)
    ASIC2.pipoclk_delay_row3.set(5)
    
    # ADC settings
    ASIC1.DHg.set(False)   # Set High gain
    ASIC1.S2D_1_b.set(0)   # Set bias current to max (min value)
    ASIC1.Ref_gen_d.set(2) # Set 'common-mode' voltage of S/H stage
    ASIC1.RefGenB.set(2)   # Don't touch

    ASIC2.DHg.set(False)   # Set High gain
    ASIC2.S2D_1_b.set(0)   # Set bias current to max (min value)
    ASIC2.Ref_gen_d.set(2) # Set 'common-mode' voltage of S/H stage
    ASIC2.RefGenB.set(2)   # Don't touch
    
    print('ASIC configured')